In [1]:
import sys
print(sys.path)
sys.path.insert(0, sys.path[0].replace('/src/jupyter_nb', ''))
print(sys.path)
import src.utils as utils
import src.myenv as myenv
import pandas as pd

['/home/marcelo/des/mg_crypto_trader/src/jupyter_nb', '/home/marcelo/anaconda3/envs/mg/lib/python310.zip', '/home/marcelo/anaconda3/envs/mg/lib/python3.10', '/home/marcelo/anaconda3/envs/mg/lib/python3.10/lib-dynload', '', '/home/marcelo/anaconda3/envs/mg/lib/python3.10/site-packages']
['/home/marcelo/des/mg_crypto_trader', '/home/marcelo/des/mg_crypto_trader/src/jupyter_nb', '/home/marcelo/anaconda3/envs/mg/lib/python310.zip', '/home/marcelo/anaconda3/envs/mg/lib/python3.10', '/home/marcelo/anaconda3/envs/mg/lib/python3.10/lib-dynload', '', '/home/marcelo/anaconda3/envs/mg/lib/python3.10/site-packages']
<built-in method __dir__ of sqlite3.Connection object at 0x7f2982268840>


In [2]:
from datetime import datetime, timedelta
import pytz

def get_start_timestamp_for_interval(interval):
	date = None
	match interval:
		case '1m':
			date = datetime.now(pytz.utc) - timedelta(days=30*9) # datetime(2023, 6, 1, 0, 0, 0, 0, pytz.utc)
		case '5m':
			date = datetime.now(pytz.utc) - timedelta(days=30*12) # datetime(2023, 1, 1, 0, 0, 0, 0, pytz.utc)
		case '15m':
			date = datetime.now(pytz.utc) - timedelta(days=365*2) # datetime(2022, 1, 1, 0, 0, 0, 0, pytz.utc)
		case '30m':
			date = datetime.now(pytz.utc) - timedelta(days=365*3) # datetime(2021, 1, 1, 0, 0, 0, 0, pytz.utc)
		case '1h':
			date = datetime.now(pytz.utc) - timedelta(days=365*6) # datetime(2017, 1, 1, 0, 0, 0, 0, pytz.utc)
	
	return int(date.timestamp() * 1000)
	

def reduce_database(interval_list = ['1m', '5m', '15m', '30m', '1h']):		
	for symbol in utils.get_symbol_list():
		for interval in interval_list:
			data_file = f'{myenv.datadir}/{symbol}/{symbol}_{interval}.dat'
			parsed_date = get_start_timestamp_for_interval(interval)
			print(data_file)
			data = utils.get_data(symbol=symbol, save_database=False, interval=interval, columns=myenv.all_klines_cols, parse_dates=False)
			data.info()
			data = data[(data['open_time'] >= parsed_date)]
			data.info()
			data.to_csv(data_file, sep=';', index=False, compression=dict(method='zip'))
	

In [3]:
reduce_database()
	



/home/marcelo/des/mg_crypto_trader/data/BTCUSDT/BTCUSDT_1m.dat
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3238425 entries, 1502942400000 to 1697761500000
Data columns (total 13 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   open_time                     int64  
 1   open                          float32
 2   high                          float32
 3   low                           float32
 4   close                         float32
 5   volume                        float32
 6   close_time                    int64  
 7   quote_asset_volume            float32
 8   number_of_trades              int16  
 9   taker_buy_base_asset_volume   float32
 10  taker_buy_quote_asset_volume  float32
 11  ignore                        int16  
 12  symbol                        object 
dtypes: float32(8), int16(2), int64(2), object(1)
memory usage: 210.0+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 388721 entries, 1674433500000 to 1697

In [44]:
data = utils.get_data(symbol='BTCUSDT', save_database=False, interval='1h', columns=myenv.all_klines_cols, parse_dates=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52441 entries, 2017-10-21 01:00:00 to 2023-10-20 01:00:00
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     52441 non-null  datetime64[ns]
 1   open                          52441 non-null  float32       
 2   high                          52441 non-null  float32       
 3   low                           52441 non-null  float32       
 4   close                         52441 non-null  float32       
 5   volume                        52441 non-null  float32       
 6   close_time                    52441 non-null  int64         
 7   quote_asset_volume            52441 non-null  float32       
 8   number_of_trades              52441 non-null  int16         
 9   taker_buy_base_asset_volume   52441 non-null  float32       
 10  taker_buy_quote_asset_volume  52441 non-null  float32      

In [45]:
import numpy as np

min_date = data['open_time'].min()
max_date = data['open_time'].max()
print(f'Data Loaded: Min date: {min_date} - Max date: {max_date}')
validate_start_data = max_date - np.timedelta64(myenv.days_to_validate_train, 'D')

print(f'Filtering all data: days_to_validate_train_data: {myenv.days_to_validate_train} days')
min_index = data[data['open_time'] < validate_start_data].tail(myenv.rows_to_train).index.min()
real_rows_to_train = data[data['open_time'] < validate_start_data].tail(myenv.rows_to_train).shape[0]
real_rols_to_validate = data[data['open_time'] >= validate_start_data].shape[0]
data = data[data.index >= min_index]

min_date = data['open_time'].min()
max_date = data['open_time'].max()

print(f'All Data Filtered: train_start_date: {min_date} - validate_start_data: {validate_start_data} - max_date: {max_date}')
print(f'rows_to_train: {real_rows_to_train} - rows_to_validate: {real_rols_to_validate} - All Data Shape: {data.shape[0]}')

Data Loaded: Min date: 2017-10-21 01:00:00 - Max date: 2023-10-20 01:00:00
Filtering all data: days_to_validate_train_data: 90 days
All Data Filtered: train_start_date: 2017-11-01 17:00:00 - validate_start_data: 2023-07-22 01:00:00 - max_date: 2023-10-20 01:00:00
rows_to_train: 50000 - rows_to_validate: 2161 - All Data Shape: 52161


In [46]:
from datetime import datetime, timedelta
print("Current time: ", datetime.now())
print(datetime.now() - timedelta(days=365))

Current time:  2023-10-19 22:55:19.226495
2022-10-19 22:55:19.226573
